In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

from IPython.display import clear_output
from datetime import datetime

In [2]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./2.item.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)

# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        import requests

        import sys
        sys.path.append('../../00.Tools')
        from crawler_configuration import get_proxy

        try:
            b = 0
            while True:

                while True:
                    b += 1
                    try:
                        resp = requests.get(input_.loc[a, 'Description Url'],
                                            proxies=get_proxy(),
                                            timeout=(10, 10))
                        break
                    except:
                        continue

                if resp.status_code == 200:
                    break

            # = = = = = = = = = = = = = = =

            from bs4 import BeautifulSoup
            from lxml import etree
            
            soup = BeautifulSoup(resp.text, 'lxml')
            html = etree.HTML(str(soup))

            # = = = = = = = = = = = = = = =

            import re
            
            list_text = html.xpath('//div[@class="x-item-description-child"]/center/table/tbody/tr[2]/td/table[2]/tbody/tr[2]/td[@class="text"]/div/descendant-or-self::strong')
            list_text = [etree.tostring(text).decode('utf-8') for text in list_text]
            list_text = [re.sub(r'(<br/>)+', '<br/>', text) for text in list_text]
            list_text = [re.sub(r'<.*?>', ' ', text.replace('<br/>', '\n')) for text in list_text]
            list_text = [re.sub(r' +', ' ', text).strip() for text in list_text]

            # = = = = = = = = = = = = = = =

            df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                     'Item Number': input_.loc[a, 'Item Number'],
                                     'Description Url': input_.loc[a, 'Description Url']}])

            for i, text in zip(range(len(list_text)), list_text):
                df_temp.loc[0, f'Text {i+1}'] = text

            output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            crawler_status = 'ok'
        except:
            df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                     'Item Number': input_.loc[a, 'Item Number'],
                                     'Description Url': input_.loc[a, 'Description Url']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        # = = = = = = = = = = = = = = =

        if work.qsize() % 1_000 == 0 and work.qsize() != 0:
            clear_output()

        print(f'''[状态：{crawler_status}，尝试次数：{b}] - {input_.loc[a, 'No']}.{input_.loc[a, 'Item Number']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(30):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
    output_correct.to_excel(f'''./4.description-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./description_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

[状态：ok，尝试次数：1] - 5278.400979701858
[剩余数量：1000] - [当前时间：17:25:41]

[状态：ok，尝试次数：1] - 5273.351655997487
[剩余数量：999] - [当前时间：17:25:41]

[状态：ok，尝试次数：1] - 5277.351484120082
[剩余数量：998] - [当前时间：17:25:41]

[状态：ok，尝试次数：1] - 5272.351655995289
[剩余数量：997] - [当前时间：17:25:41]

[状态：ok，尝试次数：1] - 5275.352619535545
[剩余数量：996] - [当前时间：17:25:41]

[状态：ok，尝试次数：1] - 5263.351655847322
[剩余数量：995] - [当前时间：17:25:41]

[状态：ok，尝试次数：1] - 5279.354230180712
[剩余数量：994] - [当前时间：17:25:41]

[状态：ok，尝试次数：1] - 5266.400270717530
[剩余数量：993] - [当前时间：17:25:41]

[状态：ok，尝试次数：1] - 5210.120778000303
[剩余数量：992] - [当前时间：17:25:42]

[状态：ok，尝试次数：1] - 5276.120712942218
[剩余数量：991] - [当前时间：17:25:42]

[状态：ok，尝试次数：1] - 5282.402366529915
[剩余数量：990] - [当前时间：17:25:42]

[状态：ok，尝试次数：1] - 5280.121297317272
[剩余数量：989] - [当前时间：17:25:42]

[状态：ok，尝试次数：1] - 5270.352212081846
[剩余数量：988] - [当前时间：17:25:42]

[状态：ok，尝试次数：1] - 5271.404384207366
[剩余数量：987] - [当前时间：17:25:42]

[状态：ok，尝试次数：1] - 5281.400264484088
[剩余数量：986] - [当前时间：17:25:42]

[状态：ok，尝试次数：1] - 5286.35